In [1]:
import os, sys
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from qqman import qqman

In [21]:
DATScan = pd.read_csv("../Imaging/DATScan_Norm.csv")
images = DATScan.iloc[:,np.r_[0,3:7]]
images = images.groupby('PATNO').first().reset_index()
images

,PATNO,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,2.99,3.43,2.328620,1.973532
1,3001,1.56,1.12,-0.246956,-0.796382
2,3002,2.13,2.28,0.255480,0.339528
3,3003,2.19,1.27,-0.118658,-0.401457
4,3004,5.09,5.30,2.949289,2.400680
...,...,...,...,...,...
990,75484,1.35,1.24,-0.765017,-1.158726
991,75505,1.90,1.66,0.065569,-0.486548
992,75524,1.36,1.76,-0.665445,0.269728
993,75537,1.46,1.03,0.065569,-1.070600


In [6]:
ppmi_info = pd.read_csv('../PPMI_data/Info files/Screening___Demographics.csv')

pca = pd.read_table('Data/QC/White.eigenvec', sep=' ', header=None)
cnames = ['FID','IID']
for i in range(1,11):
    cnames.append("PC"+str(i))
pca.columns = cnames

info_pca = pca.merge(ppmi_info, left_on=['FID'], right_on=['PATNO'])

apprdx = list()

for i in info_pca['APPRDX'].to_list():
    if(i==1):
        apprdx.append("PD")
    if(i==2):
        apprdx.append("HC")
    elif(i==3):
        apprdx.append("SWEDD")

info_pca['APPRDX2'] = apprdx
info_pca

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,REFERRAL,DECLINED,RSNDEC,EXCLUDED,RSNEXC,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV,APPRDX2
0,3000,3000,0.021562,-0.005305,0.012287,0.065969,0.006345,0.011038,-0.051736,0.020446,...,31.0,NaN,NaN,NaN,NaN,01/2011,2011-01-19 12:44:58.0,NaN,01/2011,HC
1,3001,3001,0.013209,0.016830,0.014295,-0.033689,-0.039806,0.047147,-0.026051,0.040981,...,60.0,NaN,NaN,NaN,NaN,02/2011,2011-02-10 10:43:30.0,NaN,03/2011,PD
2,3002,3002,-0.082064,-0.030599,-0.021643,-0.001498,0.025769,-0.005442,0.006985,-0.015841,...,1.0,NaN,NaN,NaN,NaN,03/2011,2011-03-04 11:04:58.0,NaN,03/2011,PD
3,3004,3004,0.023291,0.022143,-0.013579,-0.013314,-0.029992,-0.025549,-0.115247,-0.001666,...,99.0,NaN,NaN,NaN,NaN,03/2011,2011-03-30 11:43:13.0,NaN,03/2011,HC
4,3006,3006,0.008937,0.057132,0.088903,0.031578,0.002286,0.036026,-0.031903,-0.001299,...,1.0,NaN,NaN,NaN,NaN,03/2011,2011-03-31 07:21:03.0,NaN,03/2011,PD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,4124,4124,0.007515,0.030850,0.032300,0.006790,0.075679,-0.048470,0.028239,-0.062672,...,1.0,NaN,NaN,NaN,NaN,02/2013,2013-02-22 12:57:46.0,NaN,02/2013,PD
476,4125,4125,-0.015873,0.026348,-0.005282,-0.022791,-0.076218,-0.045724,-0.004484,0.017099,...,1.0,NaN,NaN,NaN,NaN,02/2013,2013-12-10 12:36:00.0,NaN,05/2013,PD
477,4126,4126,-0.006236,-0.004422,-0.054059,0.034937,0.011776,-0.098040,-0.040385,0.032559,...,1.0,NaN,NaN,NaN,NaN,03/2013,2013-03-27 08:35:42.0,NaN,03/2013,PD
478,4136,4136,0.036144,-0.015858,-0.064136,-0.033492,0.042224,-0.105760,0.021767,-0.099659,...,99.0,NaN,NaN,NaN,NaN,03/2013,2013-03-06 07:09:02.0,NaN,06/2013,PD


In [66]:
info_pheno = info_pca.merge(images, left_on=['FID'], right_on=['PATNO'])
info_pheno

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV,APPRDX2,PATNO_y,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,3000,0.021562,-0.005305,0.012287,0.065969,0.006345,0.011038,-0.051736,0.020446,...,01/2011,2011-01-19 12:44:58.0,NaN,01/2011,HC,3000,2.99,3.43,2.328620,1.973532
1,3001,3001,0.013209,0.016830,0.014295,-0.033689,-0.039806,0.047147,-0.026051,0.040981,...,02/2011,2011-02-10 10:43:30.0,NaN,03/2011,PD,3001,1.56,1.12,-0.246956,-0.796382
2,3002,3002,-0.082064,-0.030599,-0.021643,-0.001498,0.025769,-0.005442,0.006985,-0.015841,...,03/2011,2011-03-04 11:04:58.0,NaN,03/2011,PD,3002,2.13,2.28,0.255480,0.339528
3,3004,3004,0.023291,0.022143,-0.013579,-0.013314,-0.029992,-0.025549,-0.115247,-0.001666,...,03/2011,2011-03-30 11:43:13.0,NaN,03/2011,HC,3004,5.09,5.30,2.949289,2.400680
4,3006,3006,0.008937,0.057132,0.088903,0.031578,0.002286,0.036026,-0.031903,-0.001299,...,03/2011,2011-03-31 07:21:03.0,NaN,03/2011,PD,3006,2.28,2.12,0.175997,-2.400680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,4124,4124,0.007515,0.030850,0.032300,0.006790,0.075679,-0.048470,0.028239,-0.062672,...,02/2013,2013-02-22 12:57:46.0,NaN,02/2013,PD,4124,1.95,1.59,-0.273727,-0.053985
471,4125,4125,-0.015873,0.026348,-0.005282,-0.022791,-0.076218,-0.045724,-0.004484,0.017099,...,02/2013,2013-12-10 12:36:00.0,NaN,05/2013,PD,4125,2.03,2.47,0.391926,1.005487
472,4126,4126,-0.006236,-0.004422,-0.054059,0.034937,0.011776,-0.098040,-0.040385,0.032559,...,03/2013,2013-03-27 08:35:42.0,NaN,03/2013,PD,4126,1.54,1.88,-0.219232,-1.158726
473,4136,4136,0.036144,-0.015858,-0.064136,-0.033492,0.042224,-0.105760,0.021767,-0.099659,...,03/2013,2013-03-06 07:09:02.0,NaN,06/2013,PD,4136,0.68,0.57,-1.406477,-1.922162


In [30]:
info_pheno.iloc[:,0:2].to_csv("Data/QC/IDs_White.txt", header=False, index=False, sep=" ")

In [31]:
!plink --bfile Data/QC/White --keep Data/QC/IDs_White.txt --make-bed --out Data/QC/White_2

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to Data/QC/White_2.log.
Options in effect:
  --bfile Data/QC/White
  --keep Data/QC/IDs_White.txt
  --make-bed
  --out Data/QC/White_2

32768 MB RAM detected; reserving 16384 MB for main workspace.
128957 variants loaded from .bim file.
480 people (317 males, 163 females) loaded from .fam.
--keep: 475 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 475 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.996613.
128957 variants and 475 people pass filters and QC.
Note: No phenotypes present.
--make-bed to Data/QC/White_2.b

In [5]:
!plink --bfile Data/QC/White_2 --pca 10 --out Data/QC/White_2

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to Data/QC/White_2.log.
Options in effect:
  --bfile Data/QC/White_2
  --out Data/QC/White_2
  --pca 10

32768 MB RAM detected; reserving 16384 MB for main workspace.
128957 variants loaded from .bim file.
475 people (312 males, 163 females) loaded from .fam.
Using up to 11 threads (change this with --threads).
Before main variant filters, 475 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.996613.
128957 variants and 475 people pass filters and QC.
Note: No phenotypes present.
Relationship matrix calculation complete.
--pca: Results saved to Data/QC/White_2.eigenval and Data/QC/White_2.eigen

In [32]:
pheno = info_pheno[["FID","IID","APPRDX2","CAUDATE_R","CAUDATE_L","PUTAMEN_R","PUTAMEN_L"]]
pheno

,FID,IID,APPRDX2,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,3000,HC,2.99,3.43,2.328620,1.973532
1,3001,3001,PD,1.56,1.12,-0.246956,-0.796382
2,3002,3002,PD,2.13,2.28,0.255480,0.339528
3,3004,3004,HC,5.09,5.30,2.949289,2.400680
4,3006,3006,PD,2.28,2.12,0.175997,-2.400680
...,...,...,...,...,...,...,...
470,4124,4124,PD,1.95,1.59,-0.273727,-0.053985
471,4125,4125,PD,2.03,2.47,0.391926,1.005487
472,4126,4126,PD,1.54,1.88,-0.219232,-1.158726
473,4136,4136,PD,0.68,0.57,-1.406477,-1.922162


In [33]:
pheno.to_csv('GWAS2/Phenotype.txt',sep="\t", index=False)

In [67]:
age = list()

for i in range(len(info_pheno)):
    mm,yy = info_pheno.loc[i,"SITE_APRV"].split("/")
    a = int(yy) - int(info_pheno.loc[i,"BIRTHDT"])
    age.append(a)

info_pheno["AGE"] = age   

In [35]:
cov = info_pheno[["FID","IID","AGE","PC1","PC2","PC3","PC4","PC5"]]
cov.to_csv('GWAS2/Covariate.txt',sep="\t", index=False)
cov

,FID,IID,AGE,PC1,PC2,PC3,PC4,PC5
0,3000,3000,70,0.021562,-0.005305,0.012287,0.065969,0.006345
1,3001,3001,65,0.013209,0.016830,0.014295,-0.033689,-0.039806
2,3002,3002,68,-0.082064,-0.030599,-0.021643,-0.001498,0.025769
3,3004,3004,60,0.023291,0.022143,-0.013579,-0.013314,-0.029992
4,3006,3006,58,0.008937,0.057132,0.088903,0.031578,0.002286
...,...,...,...,...,...,...,...,...
470,4124,4124,71,0.007515,0.030850,0.032300,0.006790,0.075679
471,4125,4125,64,-0.015873,0.026348,-0.005282,-0.022791,-0.076218
472,4126,4126,56,-0.006236,-0.004422,-0.054059,0.034937,0.011776
473,4136,4136,56,0.036144,-0.015858,-0.064136,-0.033492,0.042224


In [37]:
col = list(pheno.columns.values)[3:7] 
col

['CAUDATE_R', 'CAUDATE_L', 'PUTAMEN_R', 'PUTAMEN_L']

In [38]:
for i in col:
    !plink --bfile DATA/QC/White_2 --pheno GWAS2/Phenotype.txt --covar GWAS2/Covariate.txt --pheno-name {i} --covar-name AGE,PC1-PC5 --linear sex --adjust --out GWAS2/{i}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to GWAS2/CAUDATE_R.log.
Options in effect:
  --adjust
  --bfile DATA/QC/White_2
  --covar GWAS2/Covariate.txt
  --covar-name AGE,PC1-PC5
  --linear sex
  --out GWAS2/CAUDATE_R
  --pheno GWAS2/Phenotype.txt
  --pheno-name CAUDATE_R

32768 MB RAM detected; reserving 16384 MB for main workspace.
128957 variants loaded from .bim file.
475 people (312 males, 163 females) loaded from .fam.
475 phenotype values present after --pheno.
Using 1 thread (no multithreaded calculations invoked).
--covar: 6 covariates loaded.
Before main variant filters, 475 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.9

In [39]:
for i in col:
    gwas = pd.read_table("GWAS2/"+i+".assoc.linear",sep="\s+")
    gwas = gwas[gwas.TEST=="ADD"]
    gwas.to_csv('GWAS2/GWAS_DATScan_'+i+'.csv',index=False)

# Datscan GWAS without SWEDD

In [48]:
pca = pd.read_table('Data/QC/White_no_swedd.eigenvec', sep=' ', header=None)
cnames = ['FID','IID']
for i in range(1,11):
    cnames.append("PC"+str(i))
pca.columns = cnames

info_pca = pca.merge(ppmi_info, left_on=['FID'], right_on=['PATNO'])

apprdx = list()

for i in info_pca['APPRDX'].to_list():
    if(i==1):
        apprdx.append("PD")
    if(i==2):
        apprdx.append("HC")
    elif(i==3):
        apprdx.append("SWEDD")

info_pca['APPRDX2'] = apprdx
info_pca

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,REFERRAL,DECLINED,RSNDEC,EXCLUDED,RSNEXC,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV,APPRDX2
0,3000,3000,0.026492,0.000678,0.035338,0.064825,0.000766,-0.041875,0.033775,0.013332,...,31.0,NaN,NaN,NaN,NaN,01/2011,2011-01-19 12:44:58.0,NaN,01/2011,HC
1,3001,3001,0.017766,0.017305,0.000688,-0.045234,-0.037821,-0.066630,0.010062,-0.054224,...,60.0,NaN,NaN,NaN,NaN,02/2011,2011-02-10 10:43:30.0,NaN,03/2011,PD
2,3002,3002,-0.084263,-0.034856,-0.023204,0.006116,0.027577,0.008214,-0.006264,0.008593,...,1.0,NaN,NaN,NaN,NaN,03/2011,2011-03-04 11:04:58.0,NaN,03/2011,PD
3,3004,3004,0.024563,0.033477,-0.003634,-0.005703,-0.042231,-0.059694,0.067412,0.017923,...,99.0,NaN,NaN,NaN,NaN,03/2011,2011-03-30 11:43:13.0,NaN,03/2011,HC
4,3006,3006,0.008511,0.056539,0.101538,0.001351,0.006814,-0.047180,-0.012988,-0.033577,...,1.0,NaN,NaN,NaN,NaN,03/2011,2011-03-31 07:21:03.0,NaN,03/2011,PD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,4124,4124,0.008077,0.029449,0.040843,-0.013005,0.074609,0.068977,0.010669,0.081488,...,1.0,NaN,NaN,NaN,NaN,02/2013,2013-02-22 12:57:46.0,NaN,02/2013,PD
432,4125,4125,-0.019947,0.029816,-0.017732,-0.017475,-0.082603,0.015329,0.025788,0.053534,...,1.0,NaN,NaN,NaN,NaN,02/2013,2013-12-10 12:36:00.0,NaN,05/2013,PD
433,4126,4126,-0.009423,0.001673,-0.037664,0.060811,-0.006292,0.034639,0.107867,0.040532,...,1.0,NaN,NaN,NaN,NaN,03/2013,2013-03-27 08:35:42.0,NaN,03/2013,PD
434,4136,4136,0.032763,-0.016046,-0.064461,-0.015332,0.030023,0.123020,0.032174,0.084627,...,99.0,NaN,NaN,NaN,NaN,03/2013,2013-03-06 07:09:02.0,NaN,06/2013,PD


In [49]:
info_pheno = info_pca.merge(images, left_on=['FID'], right_on=['PATNO'])
info_pheno

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV,APPRDX2,PATNO_y,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,3000,0.026492,0.000678,0.035338,0.064825,0.000766,-0.041875,0.033775,0.013332,...,01/2011,2011-01-19 12:44:58.0,NaN,01/2011,HC,3000,2.99,3.43,2.328620,1.973532
1,3001,3001,0.017766,0.017305,0.000688,-0.045234,-0.037821,-0.066630,0.010062,-0.054224,...,02/2011,2011-02-10 10:43:30.0,NaN,03/2011,PD,3001,1.56,1.12,-0.246956,-0.796382
2,3002,3002,-0.084263,-0.034856,-0.023204,0.006116,0.027577,0.008214,-0.006264,0.008593,...,03/2011,2011-03-04 11:04:58.0,NaN,03/2011,PD,3002,2.13,2.28,0.255480,0.339528
3,3004,3004,0.024563,0.033477,-0.003634,-0.005703,-0.042231,-0.059694,0.067412,0.017923,...,03/2011,2011-03-30 11:43:13.0,NaN,03/2011,HC,3004,5.09,5.30,2.949289,2.400680
4,3006,3006,0.008511,0.056539,0.101538,0.001351,0.006814,-0.047180,-0.012988,-0.033577,...,03/2011,2011-03-31 07:21:03.0,NaN,03/2011,PD,3006,2.28,2.12,0.175997,-2.400680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,4124,4124,0.008077,0.029449,0.040843,-0.013005,0.074609,0.068977,0.010669,0.081488,...,02/2013,2013-02-22 12:57:46.0,NaN,02/2013,PD,4124,1.95,1.59,-0.273727,-0.053985
429,4125,4125,-0.019947,0.029816,-0.017732,-0.017475,-0.082603,0.015329,0.025788,0.053534,...,02/2013,2013-12-10 12:36:00.0,NaN,05/2013,PD,4125,2.03,2.47,0.391926,1.005487
430,4126,4126,-0.009423,0.001673,-0.037664,0.060811,-0.006292,0.034639,0.107867,0.040532,...,03/2013,2013-03-27 08:35:42.0,NaN,03/2013,PD,4126,1.54,1.88,-0.219232,-1.158726
431,4136,4136,0.032763,-0.016046,-0.064461,-0.015332,0.030023,0.123020,0.032174,0.084627,...,03/2013,2013-03-06 07:09:02.0,NaN,06/2013,PD,4136,0.68,0.57,-1.406477,-1.922162


In [51]:
info_pheno.iloc[:,0:2].to_csv("Data/QC/IDs_White_no_swedd.txt", header=False, index=False, sep=" ")
!plink --bfile Data/QC/White_no_swedd --keep Data/QC/IDs_White_no_swedd.txt --make-bed --out Data/QC/White_2_no_swedd

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to Data/QC/White_2_no_swedd.log.
Options in effect:
  --bfile Data/QC/White_no_swedd
  --keep Data/QC/IDs_White_no_swedd.txt
  --make-bed
  --out Data/QC/White_2_no_swedd

32768 MB RAM detected; reserving 16384 MB for main workspace.
128957 variants loaded from .bim file.
436 people (289 males, 147 females) loaded from .fam.
--keep: 433 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 433 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.996625.
128957 variants and 433 people pass filters and QC.
Note: No phenotypes pres

In [53]:
pheno = info_pheno[["FID","IID","APPRDX2","CAUDATE_R","CAUDATE_L","PUTAMEN_R","PUTAMEN_L"]]
pheno.to_csv('GWAS2/Phenotype.txt',sep="\t", index=False)
pheno

,FID,IID,APPRDX2,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,3000,HC,2.99,3.43,2.328620,1.973532
1,3001,3001,PD,1.56,1.12,-0.246956,-0.796382
2,3002,3002,PD,2.13,2.28,0.255480,0.339528
3,3004,3004,HC,5.09,5.30,2.949289,2.400680
4,3006,3006,PD,2.28,2.12,0.175997,-2.400680
...,...,...,...,...,...,...,...
428,4124,4124,PD,1.95,1.59,-0.273727,-0.053985
429,4125,4125,PD,2.03,2.47,0.391926,1.005487
430,4126,4126,PD,1.54,1.88,-0.219232,-1.158726
431,4136,4136,PD,0.68,0.57,-1.406477,-1.922162


In [54]:
age = list()

for i in range(len(info_pheno)):
    mm,yy = info_pheno.loc[i,"SITE_APRV"].split("/")
    a = int(yy) - int(info_pheno.loc[i,"BIRTHDT"])
    age.append(a)

info_pheno["AGE"] = age   

cov = info_pheno[["FID","IID","AGE","PC1","PC2","PC3","PC4","PC5"]]
cov.to_csv('GWAS2/Covariate.txt',sep="\t", index=False)
cov

,FID,IID,AGE,PC1,PC2,PC3,PC4,PC5
0,3000,3000,70,0.026492,0.000678,0.035338,0.064825,0.000766
1,3001,3001,65,0.017766,0.017305,0.000688,-0.045234,-0.037821
2,3002,3002,68,-0.084263,-0.034856,-0.023204,0.006116,0.027577
3,3004,3004,60,0.024563,0.033477,-0.003634,-0.005703,-0.042231
4,3006,3006,58,0.008511,0.056539,0.101538,0.001351,0.006814
...,...,...,...,...,...,...,...,...
428,4124,4124,71,0.008077,0.029449,0.040843,-0.013005,0.074609
429,4125,4125,64,-0.019947,0.029816,-0.017732,-0.017475,-0.082603
430,4126,4126,56,-0.009423,0.001673,-0.037664,0.060811,-0.006292
431,4136,4136,56,0.032763,-0.016046,-0.064461,-0.015332,0.030023


In [55]:
col = list(pheno.columns.values)[3:7] 
col

['CAUDATE_R', 'CAUDATE_L', 'PUTAMEN_R', 'PUTAMEN_L']

In [56]:
for i in col:
    !plink --bfile DATA/QC/White_2_no_swedd --pheno GWAS2/Phenotype.txt --covar GWAS2/Covariate.txt --pheno-name {i} --covar-name AGE,PC1-PC5 --linear sex --adjust --out GWAS2/{i}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to GWAS2/CAUDATE_R.log.
Options in effect:
  --adjust
  --bfile DATA/QC/White_2_no_swedd
  --covar GWAS2/Covariate.txt
  --covar-name AGE,PC1-PC5
  --linear sex
  --out GWAS2/CAUDATE_R
  --pheno GWAS2/Phenotype.txt
  --pheno-name CAUDATE_R

32768 MB RAM detected; reserving 16384 MB for main workspace.
128957 variants loaded from .bim file.
433 people (286 males, 147 females) loaded from .fam.
433 phenotype values present after --pheno.
Using 1 thread (no multithreaded calculations invoked).
--covar: 6 covariates loaded.
Before main variant filters, 433 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping ra

In [57]:
for i in col:
    gwas = pd.read_table("GWAS2/"+i+".assoc.linear",sep="\s+")
    gwas = gwas[gwas.TEST=="ADD"]
    gwas.to_csv('GWAS2/GWAS_DATScan_'+i+'.csv',index=False)

# MRI GWAS WITH SWEDD

In [2]:
MRI = pd.read_csv("../Processed_Images/PPMIMERGE_20200419.csv")
MRI_unique = MRI.groupby('PATNO').first().reset_index()
MRI_unique

/Users/guglielmo/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (31,494,496,506,508,510,512,542,543,550,551,553,561,563,564) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PATNO,index,APPRDX_enrol,APPRDX_current,EVENT_ID,Age,Years_bl,GENDER,FAMHXPD_N1stDegree,FAMHXPD_N2ndDegree,...,TotalGrayVol,SupraTentorialVol,SupraTentorialVolNotVent,SupraTentorialVolNotVentVox,TD_score,PIGD_score,TD_PIGD_ratio,TD_PIGD_class,INFODT_,INFODT_bl
0,3000,651,2.0,2.0,BL,69.169,0.0,1.0,0.0,0.0,...,628301.286475,9.996803e+05,9.777723e+05,973296.0,0.000000,0.0,NaN,Indeterminate,2011-02-01,2011-02-01
1,3001,687,1.0,1.0,BL,65.161,NaN,2.0,0.0,0.0,...,668025.901257,1.138365e+06,1.094811e+06,1092459.0,0.272727,0.0,5.000000,TD,2011-03-01,NaN
2,3002,759,1.0,1.0,BL,67.581,NaN,1.0,0.0,0.0,...,585538.400359,9.026203e+05,8.838913e+05,881444.0,0.181818,0.6,0.303030,PIGD,2011-03-01,NaN
3,3003,831,1.0,1.0,BL,56.750,0.0,1.0,0.0,0.0,...,636432.509323,1.040302e+06,1.020943e+06,1019030.0,0.363636,0.4,0.909091,Indeterminate,2011-04-01,2011-04-01
4,3004,867,2.0,2.0,BL,59.414,0.0,2.0,0.0,0.0,...,640446.950904,1.074093e+06,1.056334e+06,1053446.0,0.000000,0.0,NaN,Indeterminate,2011-04-01,2011-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906,90456,69726,4.0,4.0,BL,82.505,0.0,2.0,0.0,0.0,...,635113.067329,1.123903e+06,1.030924e+06,1027319.0,0.090909,0.6,0.151515,PIGD,2013-09-01,2013-09-01
1907,91097,69762,4.0,4.0,BL,67.335,0.0,2.0,0.0,0.0,...,507261.673333,8.354743e+05,8.187873e+05,816977.0,0.000000,0.4,0.000000,PIGD,2014-07-01,2014-07-01
1908,91837,69798,4.0,4.0,BL,73.914,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,Indeterminate,2014-03-01,2014-03-01
1909,92490,69834,4.0,4.0,BL,72.504,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.090909,0.0,0.000000,TD,2015-01-01,2015-01-01


In [23]:
pca = pd.read_table('Data/QC/White_2.eigenvec', sep=' ', header=None)
cnames = ['FID','IID']
for i in range(1,11):
    cnames.append("PC"+str(i))
pca.columns = cnames

info_pca = pca.merge(ppmi_info, left_on=['FID'], right_on=['PATNO'])

apprdx = list()

for i in info_pca['APPRDX'].to_list():
    if(i==1):
        apprdx.append("PD")
    if(i==2):
        apprdx.append("HC")
    elif(i==3):
        apprdx.append("SWEDD")

info_pca['APPRDX2'] = apprdx

In [10]:
info_pheno = info_pca.merge(MRI_unique, left_on=['FID'], right_on=['PATNO'])
info_pheno

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,TotalGrayVol,SupraTentorialVol,SupraTentorialVolNotVent,SupraTentorialVolNotVentVox,TD_score,PIGD_score,TD_PIGD_ratio,TD_PIGD_class,INFODT_,INFODT_bl
0,3000,3000,0.021449,-0.005234,0.009304,0.066472,0.001175,0.015063,0.052470,0.019073,...,628301.286475,9.996803e+05,9.777723e+05,973296.0,0.000000,0.0,NaN,Indeterminate,2011-02-01,2011-02-01
1,3001,3001,0.013232,0.017714,0.011900,-0.036273,-0.034574,0.046887,0.015614,0.039156,...,668025.901257,1.138365e+06,1.094811e+06,1092459.0,0.272727,0.0,5.000000,TD,2011-03-01,NaN
2,3002,3002,-0.081716,-0.031366,-0.019860,0.000616,0.027651,-0.006289,-0.000023,-0.011180,...,585538.400359,9.026203e+05,8.838913e+05,881444.0,0.181818,0.6,0.303030,PIGD,2011-03-01,NaN
3,3004,3004,0.022711,0.021896,-0.009466,-0.014623,-0.035391,0.006098,0.114600,0.010290,...,640446.950904,1.074093e+06,1.056334e+06,1053446.0,0.000000,0.0,NaN,Indeterminate,2011-04-01,2011-04-01
4,3006,3006,0.008816,0.057796,0.087630,0.031654,0.002797,0.044745,0.020734,0.000451,...,691787.053845,1.120597e+06,1.091968e+06,1089715.0,0.090909,0.2,0.454545,PIGD,2011-04-01,2011-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,4124,4124,0.007829,0.031864,0.030776,0.007838,0.077760,-0.058105,-0.008439,-0.074725,...,559773.127620,1.166641e+06,1.118808e+06,1111258.0,1.090909,0.2,5.454545,TD,2013-04-01,NaN
471,4125,4125,-0.016362,0.026210,-0.005509,-0.027007,-0.079430,-0.044746,0.009454,0.006037,...,NaN,NaN,NaN,NaN,1.000000,0.0,1.136364,TD,2013-04-01,NaN
472,4126,4126,-0.005867,-0.004077,-0.053631,0.035073,-0.001406,-0.092460,0.058593,0.027439,...,695806.656485,1.145848e+06,1.115972e+06,1113066.0,0.545455,0.2,2.727273,TD,2013-03-01,NaN
473,4136,4136,0.036528,-0.014558,-0.060270,-0.033671,0.042000,-0.106515,0.008027,-0.108608,...,695968.316289,1.078311e+06,1.053845e+06,1051472.0,0.181818,0.6,0.303030,PIGD,2013-03-01,2013-03-01


In [11]:
pheno = info_pheno[["FID","IID","APPRDX2","rh_parahippocampal_area","lh_parahippocampal_area","rh_parahippocampal_volume","lh_parahippocampal_volume"]]
pheno.fillna("NA",inplace=True)
pheno.to_csv('GWAS2/MRI/Phenotype_With_SWEDD.txt',sep="\t", index=False)
pheno

/Users/guglielmo/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,FID,IID,APPRDX2,rh_parahippocampal_area,lh_parahippocampal_area,rh_parahippocampal_volume,lh_parahippocampal_volume
0,3000,3000,HC,723,680,2392,2063
1,3001,3001,PD,721,746,2053,2129
2,3002,3002,PD,606,503,2233,1612
3,3004,3004,HC,662,696,2127,2205
4,3006,3006,PD,693,837,2623,3134
...,...,...,...,...,...,...,...
470,4124,4124,PD,821,819,1582,2079
471,4125,4125,PD,NA,NA,NA,NA
472,4126,4126,PD,715,789,2091,2298
473,4136,4136,PD,843,795,2446,2321


In [16]:
age = list()
for i in range(len(info_pheno)):
    mm,yy = info_pheno.loc[i,"SITE_APRV"].split("/")
    a = int(yy) - int(info_pheno.loc[i,"BIRTHDT_x"])
    age.append(a)

info_pheno["AGE"] = age   

In [17]:
cov = info_pheno[["FID","IID","eTIV","AGE","PC1","PC2","PC3","PC4","PC5"]]
cov.fillna("NA",inplace=True)
cov.to_csv('GWAS2/MRI/Covariate_With_SWEDD.txt',sep="\t", index=False)
cov

/Users/guglielmo/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,FID,IID,eTIV,AGE,PC1,PC2,PC3,PC4,PC5
0,3000,3000,1.56977e+06,70,0.021449,-0.005234,0.009304,0.066472,0.001175
1,3001,3001,1.71515e+06,65,0.013232,0.017714,0.011900,-0.036273,-0.034574
2,3002,3002,1.35646e+06,68,-0.081716,-0.031366,-0.019860,0.000616,0.027651
3,3004,3004,1.61768e+06,60,0.022711,0.021896,-0.009466,-0.014623,-0.035391
4,3006,3006,1.6439e+06,58,0.008816,0.057796,0.087630,0.031654,0.002797
...,...,...,...,...,...,...,...,...,...
470,4124,4124,1.55545e+06,71,0.007829,0.031864,0.030776,0.007838,0.077760
471,4125,4125,NA,64,-0.016362,0.026210,-0.005509,-0.027007,-0.079430
472,4126,4126,1.8287e+06,56,-0.005867,-0.004077,-0.053631,0.035073,-0.001406
473,4136,4136,1.65036e+06,56,0.036528,-0.014558,-0.060270,-0.033671,0.042000


In [18]:
col = list(pheno.columns.values)[3:7] 
col

['rh_parahippocampal_area',
 'lh_parahippocampal_area',
 'rh_parahippocampal_volume',
 'lh_parahippocampal_volume']

In [20]:
for i in col:
    !plink --bfile DATA/QC/White_2 --pheno GWAS2/MRI/Phenotype_With_SWEDD.txt --covar GWAS2/MRI/Covariate_With_SWEDD.txt --pheno-name {i} --covar-name eTIV,AGE,PC1-PC5 --linear sex --adjust --out GWAS2/MRI/{i+"_With_SWEDD"}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to GWAS2/MRI/rh_parahippocampal_area_With_SWEDD.log.
Options in effect:
  --adjust
  --bfile DATA/QC/White_2
  --covar GWAS2/MRI/Covariate_With_SWEDD.txt
  --covar-name eTIV,AGE,PC1-PC5
  --linear sex
  --out GWAS2/MRI/rh_parahippocampal_area_With_SWEDD
  --pheno GWAS2/MRI/Phenotype_With_SWEDD.txt
  --pheno-name rh_parahippocampal_area

32768 MB RAM detected; reserving 16384 MB for main workspace.
128957 variants loaded from .bim file.
475 people (312 males, 163 females) loaded from .fam.
435 phenotype values present after --pheno.
Using 1 thread (no multithreaded calculations invoked).
--covar: 7 covariates loaded.
40 people had missing value(s).
Before main variant filters, 475 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464

In [21]:
for i in col:
    gwas = pd.read_table("GWAS2/MRI/"+i+"_With_SWEDD.assoc.linear",sep="\s+")
    gwas = gwas[gwas.TEST=="ADD"]
    gwas.to_csv('GWAS2/MRI/GWAS_MRI_'+i+'_With_SWEDD.csv',index=False)

# MRI GWAS WITHOUTH SWEDD

In [24]:
!plink --bfile Data/QC/White_2_no_swedd --pca 10 --out Data/QC/White_2_no_swedd

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to Data/QC/White_2_no_swedd.log.
Options in effect:
  --bfile Data/QC/White_2_no_swedd
  --out Data/QC/White_2_no_swedd
  --pca 10

32768 MB RAM detected; reserving 16384 MB for main workspace.
128957 variants loaded from .bim file.
433 people (286 males, 147 females) loaded from .fam.
Using up to 11 threads (change this with --threads).
Before main variant filters, 433 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.996625.
128957 variants and 433 people pass filters and QC.
Note: No phenotypes present.
Relationship matrix calculation complete.
--pca: Results saved to Data/QC/White_2_no_swed

In [27]:
pca = pd.read_table('Data/QC/White_2_no_swedd.eigenvec', sep=' ', header=None)
cnames = ['FID','IID']
for i in range(1,11):
    cnames.append("PC"+str(i))
pca.columns = cnames

info_pca = pca.merge(ppmi_info, left_on=['FID'], right_on=['PATNO'])

apprdx = list()

for i in info_pca['APPRDX'].to_list():
    if(i==1):
        apprdx.append("PD")
    if(i==2):
        apprdx.append("HC")
    elif(i==3):
        apprdx.append("SWEDD")

info_pca['APPRDX2'] = apprdx

info_pheno = info_pca.merge(MRI_unique, left_on=['FID'], right_on=['PATNO'])
age = list()
for i in range(len(info_pheno)):
    mm,yy = info_pheno.loc[i,"SITE_APRV"].split("/")
    a = int(yy) - int(info_pheno.loc[i,"BIRTHDT_x"])
    age.append(a)

info_pheno["AGE"] = age   
info_pheno

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,SupraTentorialVol,SupraTentorialVolNotVent,SupraTentorialVolNotVentVox,TD_score,PIGD_score,TD_PIGD_ratio,TD_PIGD_class,INFODT_,INFODT_bl,AGE
0,3000,3000,0.026252,0.000831,0.034813,0.066445,0.005376,-0.039601,0.039440,0.011572,...,9.996803e+05,9.777723e+05,973296.0,0.000000,0.0,NaN,Indeterminate,2011-02-01,2011-02-01,70
1,3001,3001,0.017288,0.016918,0.000809,-0.046732,0.036114,-0.066129,0.008962,-0.059123,...,1.138365e+06,1.094811e+06,1092459.0,0.272727,0.0,5.000000,TD,2011-03-01,NaN,65
2,3002,3002,-0.083952,-0.036270,-0.023529,0.008239,-0.028558,0.007373,-0.004357,0.005666,...,9.026203e+05,8.838913e+05,881444.0,0.181818,0.6,0.303030,PIGD,2011-03-01,NaN,68
3,3004,3004,0.024155,0.033869,-0.003613,-0.008359,0.043702,-0.057734,0.070331,0.013879,...,1.074093e+06,1.056334e+06,1053446.0,0.000000,0.0,NaN,Indeterminate,2011-04-01,2011-04-01,60
4,3006,3006,0.008186,0.057054,0.101822,0.001634,-0.006341,-0.047876,-0.016801,-0.033359,...,1.120597e+06,1.091968e+06,1089715.0,0.090909,0.2,0.454545,PIGD,2011-04-01,2011-04-01,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,4124,4124,0.008458,0.030614,0.040836,-0.011779,-0.077894,0.065532,0.009551,0.081294,...,1.166641e+06,1.118808e+06,1111258.0,1.090909,0.2,5.454545,TD,2013-04-01,NaN,71
429,4125,4125,-0.020655,0.029306,-0.017984,-0.021871,0.082911,0.017845,0.023150,0.058820,...,NaN,NaN,NaN,1.000000,0.0,1.136364,TD,2013-04-01,NaN,64
430,4126,4126,-0.008676,0.002931,-0.037925,0.059620,0.013533,0.038492,0.110887,0.040737,...,1.145848e+06,1.115972e+06,1113066.0,0.545455,0.2,2.727273,TD,2013-03-01,NaN,56
431,4136,4136,0.033337,-0.014685,-0.064613,-0.017093,-0.033616,0.121496,0.029545,0.085788,...,1.078311e+06,1.053845e+06,1051472.0,0.181818,0.6,0.303030,PIGD,2013-03-01,2013-03-01,56


In [28]:
pheno = info_pheno[["FID","IID","APPRDX2","rh_parahippocampal_area","lh_parahippocampal_area","rh_parahippocampal_volume","lh_parahippocampal_volume"]]
pheno.fillna("NA",inplace=True)
pheno.to_csv('GWAS2/MRI/Phenotype_Without_SWEDD.txt',sep="\t", index=False)
pheno

/Users/guglielmo/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,FID,IID,APPRDX2,rh_parahippocampal_area,lh_parahippocampal_area,rh_parahippocampal_volume,lh_parahippocampal_volume
0,3000,3000,HC,723,680,2392,2063
1,3001,3001,PD,721,746,2053,2129
2,3002,3002,PD,606,503,2233,1612
3,3004,3004,HC,662,696,2127,2205
4,3006,3006,PD,693,837,2623,3134
...,...,...,...,...,...,...,...
428,4124,4124,PD,821,819,1582,2079
429,4125,4125,PD,NA,NA,NA,NA
430,4126,4126,PD,715,789,2091,2298
431,4136,4136,PD,843,795,2446,2321


In [29]:
cov = info_pheno[["FID","IID","eTIV","AGE","PC1","PC2","PC3","PC4","PC5"]]
cov.fillna("NA",inplace=True)
cov.to_csv('GWAS2/MRI/Covariate_Without_SWEDD.txt',sep="\t", index=False)
cov

/Users/guglielmo/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,FID,IID,eTIV,AGE,PC1,PC2,PC3,PC4,PC5
0,3000,3000,1.56977e+06,70,0.026252,0.000831,0.034813,0.066445,0.005376
1,3001,3001,1.71515e+06,65,0.017288,0.016918,0.000809,-0.046732,0.036114
2,3002,3002,1.35646e+06,68,-0.083952,-0.036270,-0.023529,0.008239,-0.028558
3,3004,3004,1.61768e+06,60,0.024155,0.033869,-0.003613,-0.008359,0.043702
4,3006,3006,1.6439e+06,58,0.008186,0.057054,0.101822,0.001634,-0.006341
...,...,...,...,...,...,...,...,...,...
428,4124,4124,1.55545e+06,71,0.008458,0.030614,0.040836,-0.011779,-0.077894
429,4125,4125,NA,64,-0.020655,0.029306,-0.017984,-0.021871,0.082911
430,4126,4126,1.8287e+06,56,-0.008676,0.002931,-0.037925,0.059620,0.013533
431,4136,4136,1.65036e+06,56,0.033337,-0.014685,-0.064613,-0.017093,-0.033616


In [31]:
for i in col:
    !plink --bfile DATA/QC/White_2_no_swedd --pheno GWAS2/MRI/Phenotype_Without_SWEDD.txt --covar GWAS2/MRI/Covariate_Without_SWEDD.txt --pheno-name {i} --covar-name eTIV,AGE,PC1-PC5 --linear sex --adjust --out GWAS2/MRI/{i+"_Without_SWEDD"}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to GWAS2/MRI/rh_parahippocampal_area_Without_SWEDD.log.
Options in effect:
  --adjust
  --bfile DATA/QC/White_2_no_swedd
  --covar GWAS2/MRI/Covariate_Without_SWEDD.txt
  --covar-name eTIV,AGE,PC1-PC5
  --linear sex
  --out GWAS2/MRI/rh_parahippocampal_area_Without_SWEDD
  --pheno GWAS2/MRI/Phenotype_Without_SWEDD.txt
  --pheno-name rh_parahippocampal_area

32768 MB RAM detected; reserving 16384 MB for main workspace.
128957 variants loaded from .bim file.
433 people (286 males, 147 females) loaded from .fam.
398 phenotype values present after --pheno.
Using 1 thread (no multithreaded calculations invoked).
--covar: 7 covariates loaded.
35 people had missing value(s).
Before main variant filters, 433 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536

In [33]:
for i in col:
    gwas = pd.read_table("GWAS2/MRI/"+i+"_Without_SWEDD.assoc.linear",sep="\s+")
    gwas = gwas[gwas.TEST=="ADD"]
    gwas.to_csv('GWAS2/MRI/GWAS_MRI_'+i+'_Without_SWEDD.csv',index=False)